# Assignment #9 - Data Gathering and Warehousing - DSSA-5102

Instructor: Melissa Laurino</br>
Spring 2024</br>

Name: Udoy Chowdhury
</br>
Date: 2024-03-21
<br>
<br>
**At this time in the semester:** <br>
--We have explored a dataset. <br>
--We have cleaned our dataset. <br>
--We created a Github account with a repository for this class and included a Metadata file about our data. <br>
--We introduced general and more advanced SQL syntax, queries, and applications in R.<br>
--Created our own database from scratch in R using libraries RMariaDB and RSQLite <br>
--Populated our databases we created from scratch
--Created a visual enhanced entity relationship diagram for our database
<br>

At this point, we have discussed all major statements used with SQL, but the possibilities are endless when it comes to data! Below we will explore some miscellaneous statements and tools that may be useful with your database.<br>

<br>

Read Chapter 7 & 10 in Getting Started with SQL by Thomas Nield available on Blackboard. <br>
A quick reference for SQL commands: https://www.w3schools.com/sql/default.asp <br>

Review the powerpoint and other readings specified on Blackboard in the Discussion Board.<br>

In the event your database does not meet the requirements below to answer the question, please use the database provided in Assignment #4 and #5. Remember to credit your data source.

We used two R packages to create two separate databases for practice in our previous assignment. Feel free to use your preferred package for the exploration below. Now that all of our data is loaded onto the MySQL Workbench server, you can even take the assignment a step further and complete it all within SQL without Jupyter Notebook/RMD/Visual Studio Code.<br>

Follow the instructions below to complete the assignment. Be sure to comment **all** code and answer **all** questions in markdown for full credit. The cells below are the minimum requirement, but exploring larger queries are encouraged.<br>

**Data origin:**


In [2]:
# Handling Data
import pandas as pd
# MySQL interaction
import mysql.connector
import pymysql
# Loading Data
from sqlalchemy import create_engine
# Create a DB file
import sqlite3

# Get rid of display restrictions
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

In [3]:
# Define our variables. We set these during our first class in our technology set up. 
password = 'chowdhury626'
username = 'root'
port = 3306
database_name = 'male_fifa_players'
host = 'localhost'

# Connect to the database
con = pymysql.connect(host=host, user=username, password=password, database=database_name)

# Create cursor
cursor = con.cursor()

# Specify the database
cursor.execute(f"USE {database_name}")

0

In [7]:
#List the table names of the database.
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()

List names
for table in tables:
    print(table[0])


SyntaxError: invalid syntax (3162521087.py, line 5)

In [37]:
# Test Queries
table_name = 'player_info'
query = f"SELECT * FROM {table_name} where player_id = 123454321"

results = pd.read_sql(query, con)

display(results.head())

,player_id,fifa_version,short_name,long_name,player_positions,overall,potential,value_eur,value_dol,wage_eur,wage_dol,age,dob,height_cm,height_in,weight_kg,weight_lbs,club_team_id,club_name,league_id,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,work_rate,body_type,release_clause_eur,release_clause_dol,player_tags,player_traits
0,123454321,24,U.Chowdhury,Udoy Chowdhury,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


#### CASE Statements
Case statements are similar to the if-then-else statements in programming. The data that meets the case statements in the database will be returned. You start a CASE statement with the word CASE and conclude it with an END. Between those keywords, you specify each condition with a WHEN [condition] THEN [value], where the condition and the values are specified by you.

Nield, Thomas. Getting Started with SQL (p. 71). O'Reilly Media. Kindle Edition. <br>
<br>
Write your question you are answering with your data query. <br>
<br>
**Question:** What foot are each player, are they ambidextrous?
<br>
**What tables are we joining?:** Player Info and Player Attributes

In [26]:
#Write a query that includes a CASE statement and JOIN your results between tables that answer your question.
#Remove the example question and insert yours.

query = """
SELECT 
    pi.short_name,
    pi.club_name,
    pa.player_positions,
    CASE 
        WHEN pi.preferred_foot = 'Left' THEN 'Prefers Left'
        WHEN pi.preferred_foot = 'Right' THEN 'Prefers Right'
        ELSE 'Ambidextrous'
    END AS foot_preference
FROM 
    player_info pi
JOIN 
    player_attributes pa ON pi.player_id = pa.player_id AND pi.fifa_version = pa.fifa_version
WHERE 
    pi.club_name IS NOT NULL
    AND pi.fifa_version = 24
ORDER BY 
    pi.club_name, pa.player_positions;
"""

results = pd.read_sql(query, con)

display(results.head())


,short_name,club_name,player_positions,foot_preference
0,M. Ćoćić,1860 München,"CAM, CF",Prefers Left
1,E. Zejnullahu,1860 München,"CAM, CM, RM",Prefers Right
2,M. Starke,1860 München,"CAM, CM, ST",Prefers Left
3,A. Vrenezi,1860 München,"CAM, LM",Prefers Right
4,J. Verlaat,1860 München,CB,Prefers Right


**CASE STATEMENT**
<br>
Write a second CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** Are players at their peak, on their way to become a rising star, or a veteran?
<br>
**What tables are we joining?:** Player Info and Player Attributes

In [24]:
#Write a query that includes a CASE statement and JOIN your results between tables that answer your question.

query = """
SELECT 
    pi.short_name,
    pi.age,
    pa.overall,
    CASE 
        WHEN pi.preferred_foot = 'Left' THEN 'Prefers Left'
        WHEN pi.preferred_foot = 'Right' THEN 'Prefers Right'
        ELSE 'Ambidextrous'
    END AS foot_preference,
    CASE
        WHEN pi.age <= 22 THEN 'Rising Star'
        WHEN pi.age BETWEEN 23 AND 30 THEN 'At Peak'
        ELSE 'Veteran'
    END AS career_stage,
    CASE 
        WHEN pi.age <= 22 THEN pa.overall * 1.05 -- Increase by 5% for rising stars
        WHEN pi.age > 30 THEN pa.overall * 0.95 -- Decrease by 5% for veterans
        ELSE pa.overall -- No change for players at peak
    END AS adjusted_overall
FROM 
    player_info pi
JOIN 
    player_attributes pa ON pi.player_id = pa.player_id AND pi.fifa_version = pa.fifa_version
WHERE pi.fifa_version = 24
ORDER BY 
    adjusted_overall DESC;
"""

results = pd.read_sql(query, con)

display(results.head())


,short_name,age,overall,foot_preference,career_stage,adjusted_overall
0,E. Haaland,22,91,Prefers Left,Rising Star,95.55
1,Vini Jr.,22,89,Prefers Right,Rising Star,93.45
2,K. Mbappé,24,91,Prefers Right,At Peak,91.00
3,B. Saka,21,86,Prefers Left,Rising Star,90.30
4,K. Kvaratskhelia,22,86,Prefers Right,Rising Star,90.30


**CASE STATEMENT**
<br>
Write a third CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** What kind of skill does each player have?
<br>
**What tables are we joining?:** Player Info and Player Attributes and Player Alternate Positions

In [27]:
#Write a query that includes a CASE statement and JOIN your results between tables that answer your question.

query = """
SELECT 
    pi.short_name,
    pi.age,
    pa.overall,
    CASE 
        WHEN pi.preferred_foot = 'Left' THEN 'Prefers Left'
        WHEN pi.preferred_foot = 'Right' THEN 'Prefers Right'
        ELSE 'Ambidextrous'
    END AS foot_preference,
    pap.player_positions AS alternate_positions,
    pi.skill_moves,
    CASE
        WHEN pi.skill_moves >= 4 THEN 'High Skill'
        WHEN pi.skill_moves > 1 AND pi.skill_moves < 4 THEN 'Average Skill'
        WHEN pi.skill_moves < 2 THEN 'No Skill'
        ELSE 'Other'
    END AS skill_category,
    CASE
        WHEN pi.age <= 22 THEN 'Rising Star'
        WHEN pi.age BETWEEN 23 AND 30 THEN 'At Peak'
        ELSE 'Veteran'
    END AS career_stage,
    CASE 
        WHEN pi.age <= 22 THEN pa.overall * 1.05 -- Increase by 5% for rising stars
        WHEN pi.age > 30 THEN pa.overall * 0.95 -- Decrease by 5% for veterans
        ELSE pa.overall -- No change for players at peak
    END AS adjusted_overall
FROM 
    player_alternate_positions pap
JOIN 
    player_info pi ON pap.player_id = pi.player_id AND pap.fifa_version = pi.fifa_version
JOIN 
    player_attributes pa ON pi.player_id = pa.player_id AND pi.fifa_version = pa.fifa_version
WHERE 
    pi.skill_moves IS NOT NULL
    AND pi.fifa_version = 24
ORDER BY 
    pi.skill_moves DESC, pap.player_positions;
"""

results = pd.read_sql(query, con)

display(results.head())

,short_name,age,overall,foot_preference,alternate_positions,skill_moves,skill_category,career_stage,adjusted_overall
0,L. Acosta,29,77,Prefers Right,"CAM, CF",5,High Skill,At Peak,77.0
1,M. Ødegaard,24,87,Prefers Left,"CAM, CM",5,High Skill,At Peak,87.0
2,Lucas Paquetá,25,81,Prefers Left,"CAM, CM",5,High Skill,At Peak,81.0
3,J. Musiala,20,86,Prefers Right,"CAM, CM, LM",5,High Skill,Rising Star,90.3
4,Igor Coronado,30,78,Prefers Right,"CAM, CM, LM",5,High Skill,At Peak,78.0


**NULL**
<br>
As with all data, NULL values are fields with no data. Null data can be useful to know with the INSERT INTO statement below.

Find the NULL data within your database. Write your question you are answering with your data query. <br>
<br>
**Example Question:**  What data contains no entry for player_tags?
<br>

In [34]:
#Replace the Example Question with your own and query the data for your question.
#Write a query that includes a CASE statement and JOIN your results between tables that answer your question.

query = """
SELECT *
FROM 
    player_info
WHERE 
    player_tags is NULL
"""

results = pd.read_sql(query, con)

display(results.head())

,player_id,fifa_version,short_name,long_name,player_positions,overall,potential,value_eur,value_dol,wage_eur,wage_dol,age,dob,height_cm,height_in,weight_kg,weight_lbs,club_team_id,club_name,league_id,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,work_rate,body_type,release_clause_eur,release_clause_dol,player_tags,player_traits


**INSERT INTO**
<br>
You can insert new records into a table as needed using the INSERT INTO statement. If you choose to populate a table with certain records and not others, the rest of the fields will remain empty/NULL.
<br>
For INSERT INTO, we are not querying the database, instead we are ADDING to it. We do not need to use dbGetQuery(), but instead, dbExecute()!
<br><br>
**Objective:** Add the crew members of a future date to the database
<br>
**What table(s) are we adding a record to?** crew

In [36]:
#Use INSERT INTO to manually insert data. 

cursor.execute("""
    INSERT INTO player_info (player_id, fifa_version, short_name, long_name) 
    VALUES (%s, %s, %s, %s)
""", (123454321, 24, 'U.Chowdhury', 'Udoy Chowdhury'))

#It's okay if you do not execute your code if you do not want to add any data to your database.

1

**MIN() and MAX()**
<br>
You can use these statements alone or in combination with the CASE statemts above. Find the minimum and maximum of a numerical column.
<br><br>
**Question:** 
<br>
**What table(s) are we analyzing?** 

In [4]:
#Create a simple query to find the MIN() of a value and the MAX() of a value()
#They do not have to be in the same query yet.

query = """
SELECT 
    MIN(age) AS MinAge, 
    MAX(age) AS MaxAge
FROM 
    player_info
WHERE 
    fifa_version = 24;
"""

results = pd.read_sql(query, con)

display(results.head())


,MinAge,MaxAge
0,16,42


Combine CASE statement with Min() and Max() for a more detailed query of your data:
<br><br>
**Question:** What was the lowest and highest temperature an endangered Leatherback sea turtle and Loggerhead sea turtle was observed and was the water considered warm or cold?
<br>
**What table(s) are we adding a record to?** crew

In [16]:
#Combine the CASE statement with MIN() and MAX() for a more detailed query of your data.
#You can use one of the same CASE statements as above, or create a new one that relates to your MIN and MAX.

query = """
SELECT 
    short_name,
    overall,
    overall - (SELECT MIN(overall) FROM player_info WHERE fifa_version = '24') AS DifferenceFromMin,
    (SELECT MAX(overall) FROM player_info WHERE fifa_version = '24') - overall AS DifferenceFromMax,
    CASE 
        WHEN overall < 66 THEN 'Lowest Rated'
        WHEN overall > 84 THEN 'Highest Rated'
        ELSE 'Average Rated'
    END AS RatingCategory
FROM 
    player_info 
WHERE 
    fifa_version = '24'
ORDER BY 
    overall;
"""

results = pd.read_sql(query, con)

display(results.head(15))


,short_name,overall,DifferenceFromMin,DifferenceFromMax,RatingCategory
0,D. Sawhney,47,0,44,Lowest Rated
1,C. Remtluanga,47,0,44,Lowest Rated
2,Jin Liangkuan,47,0,44,Lowest Rated
3,Yin Jie,47,0,44,Lowest Rated
4,Wu Yuhang,47,0,44,Lowest Rated
5,M. Mewlan,47,0,44,Lowest Rated
6,Lyu Jiaqiang,47,0,44,Lowest Rated
7,S. Telem,47,0,44,Lowest Rated
8,Ye Daoxin,47,0,44,Lowest Rated
9,A. Singh,47,0,44,Lowest Rated


**AVG**<br>
AVG() will take the average of a numeric field.

**Question:** 
<br>

In [17]:
#Write a query to find the average of a numerical column.      

query = """
SELECT 
    fifa_version,
    AVG(overall) AS AverageOverallRating
FROM 
    player_info
GROUP BY 
    fifa_version
ORDER BY 
    fifa_version;
"""

results = pd.read_sql(query, con)

display(results.head(15))

,fifa_version,AverageOverallRating
0,15,63.9655
1,16,65.4747
2,17,66.3215
3,18,66.4412
4,19,66.3487
5,20,66.3827
6,21,65.8048
7,22,65.9407
8,23,65.9876
9,24,66.0060


Although this query gives us a quick answer, as a stand alone data frame, it is not that useful. We can add it to our longer query above.

In [18]:
#Add the AVG() aggregate function to your CASE query above that includes the MIN() and MAX()


query = """
SELECT 
    short_name,
    overall,
    overall - (SELECT MIN(overall) FROM player_info WHERE fifa_version = '24') AS DifferenceFromMin,
    (SELECT MAX(overall) FROM player_info WHERE fifa_version = '24') - overall AS DifferenceFromMax,
    (SELECT AVG(overall) FROM player_info WHERE fifa_version = '24') AS AverageOverall,
    CASE 
        WHEN overall < 66 THEN 'Lowest Rated'
        WHEN overall > 84 THEN 'Highest Rated'
        ELSE 'Average Rated'
    END AS RatingCategory
FROM 
    player_info 
WHERE 
    fifa_version = '24'
ORDER BY 
    overall;
"""

results = pd.read_sql(query, con)

display(results.head(15))



,short_name,overall,DifferenceFromMin,DifferenceFromMax,AverageOverall,RatingCategory
0,D. Sawhney,47,0,44,66.006,Lowest Rated
1,C. Remtluanga,47,0,44,66.006,Lowest Rated
2,Jin Liangkuan,47,0,44,66.006,Lowest Rated
3,Yin Jie,47,0,44,66.006,Lowest Rated
4,Wu Yuhang,47,0,44,66.006,Lowest Rated
5,M. Mewlan,47,0,44,66.006,Lowest Rated
6,Lyu Jiaqiang,47,0,44,66.006,Lowest Rated
7,S. Telem,47,0,44,66.006,Lowest Rated
8,Ye Daoxin,47,0,44,66.006,Lowest Rated
9,A. Singh,47,0,44,66.006,Lowest Rated


Now we are starting to get multiple values that we can save any time as a .csv if needed to access later.

In [19]:
#Save your new result set as .csv file!

# Assuming 'results' holds your DataFrame from the query
results.to_csv('manupilated_fifa_24.csv', index=False)

**Aliases (AS)**
<br>
You can abbreviate your code to make it more visually appealing...or more confusing? :) <br>
<br>
Examples:<br>
FROM table_name t<br>
FROM table_name AS t<br>
<br>
**Objective:** Use abbreviations or aliases for all tables for the same code you wrote above. Be sure to obtain the same result set.

In [20]:
#Rewrite your CASE/MIN/MAX/AVG code to include aliases if you have not done so already:


#Write a query that includes a CASE statement and JOIN your results between tables that answer your question.

query = """
SELECT 
    pi.short_name,
    pi.age,
    pa.overall,
    CASE 
        WHEN pi.preferred_foot = 'Left' THEN 'Prefers Left'
        WHEN pi.preferred_foot = 'Right' THEN 'Prefers Right'
        ELSE 'Ambidextrous'
    END AS foot_preference,
    pap.player_positions AS alternate_positions,
    pi.skill_moves,
    CASE
        WHEN pi.skill_moves >= 4 THEN 'High Skill'
        WHEN pi.skill_moves > 1 AND pi.skill_moves < 4 THEN 'Average Skill'
        WHEN pi.skill_moves < 2 THEN 'No Skill'
        ELSE 'Other'
    END AS skill_category,
    CASE
        WHEN pi.age <= 22 THEN 'Rising Star'
        WHEN pi.age BETWEEN 23 AND 30 THEN 'At Peak'
        ELSE 'Veteran'
    END AS career_stage,
    CASE 
        WHEN pi.age <= 22 THEN pa.overall * 1.05 -- Increase by 5% for rising stars
        WHEN pi.age > 30 THEN pa.overall * 0.95 -- Decrease by 5% for veterans
        ELSE pa.overall -- No change for players at peak
    END AS adjusted_overall
FROM 
    player_alternate_positions pap
JOIN 
    player_info pi ON pap.player_id = pi.player_id AND pap.fifa_version = pi.fifa_version
JOIN 
    player_attributes pa ON pi.player_id = pa.player_id AND pi.fifa_version = pa.fifa_version
WHERE 
    pi.skill_moves IS NOT NULL
    AND pi.fifa_version = 24
ORDER BY 
    pi.skill_moves DESC, pap.player_positions;
"""

results = pd.read_sql(query, con)

display(results.head())





,short_name,age,overall,foot_preference,alternate_positions,skill_moves,skill_category,career_stage,adjusted_overall
0,L. Acosta,29,77,Prefers Right,"CAM, CF",5,High Skill,At Peak,77.0
1,M. Ødegaard,24,87,Prefers Left,"CAM, CM",5,High Skill,At Peak,87.0
2,Lucas Paquetá,25,81,Prefers Left,"CAM, CM",5,High Skill,At Peak,81.0
3,J. Musiala,20,86,Prefers Right,"CAM, CM, LM",5,High Skill,Rising Star,90.3
4,Igor Coronado,30,78,Prefers Right,"CAM, CM, LM",5,High Skill,At Peak,78.0


**DELETE** ~Caution!~
<br>
You can delete all records from specific tables or set a criteria to delete certain values or NULL values without deleting the table itself. It is okay if you do not execute the code if you have completed all data cleaning steps earlier in the semester.<br>
<br>
If you created autoincrement IDs for any of your data, it is recommended to use TRUNCATE TABLE instead, used the same way. The ID's will automatically reset if needed.<br>
<br>
**Example Objective:** Delete all data from the water_variables table where tide variables were not recorded before 2016.
<br>
**What table(s) are we deleting records from?** water_variables

In [11]:
#Replace the above info with your data. 
#You can write the code, but do not execute if you do not want to delete any of your data.

cursor.execute("""
DELETE FROM player_info WHERE short_name = 'Chowdhury';
""")

We can also delete entire tables in MySQL workbench by manually right clicking on the table and DROP TABLE. <br>
MySQL Workbench will prompt you to review the SQL syntax before dropping the table.<br>
The syntax is simple:<br>
DROP TABLE table_name<br>
<br>
We can also use dbExecute(con, DROP TABLE) from Jupyter Notebook/Rmarkdown/Visual Studio. You do not have to execute this code in this notebook if you do not want to delete a table.

In [33]:
#Close the database connection :)
dbDisconnect(con)

Warning message:
“Already disconnected”


**STOP**<br>
Before you submit, did you comment all your code?<br>
Did you answer all of the questions in the markdown cells?<br>
Did you rename the file and write your name at the top of the .pynb?<br>
Attach the .csv file you created with your Blackboard submission. 